# Tutorial 1: Bending of a Cantilever Beam (FEM vs Analytical)

Importing required libraries

In [22]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [23]:
# for saving files later
tutorials_dir = Path.cwd()

Initialization of the problemm and setting up the geometry

In [24]:
# Set up a 2D cantilever beam under a uniform downward traction on the top edge.
# The left edge (x=0) is clamped. For small deflection, compare the tip displacement
# at x=L, y=H/2 with the Euler–Bernoulli beam formula:
# For a cantilever of length L, height H, uniform load q, plane strain,
#   Material:
#         E = 100000, nu = 0.3.
#     Derived:
#         mu    = E/(2*(1+nu))
#         kappa = E/(3*(1-2*nu))
    
#     Analytical tip deflection for a cantilever beam under uniform load q is:
#         w(L) = q*L^4 / (8*E*I),   I = H^3/12.


# --- Beam geometry ---
L = 10.0   # length in x
H = 1.0    # height in y
nx = 30    # number of elements along length
ny = 3     # number of elements along height

ele_type = "D2_nn6_tri"  # 2D, 3-node triangle (quad)
ndof = 2                  # 2 DOFs per node (x, y)

Mesh generation and fixing a boundary

In [25]:
# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
# coords: shape (n_nodes, 2)
# connect: shape (n_nodes_per_elem, n_elems)

# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

# 1) Clamp the left edge: fix x- and y-displacements = 0
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

# 2) Uniform downward traction on the top edge (y=H)
# Let q be negative in the y-direction
q = -1  # load per unit length in x
# For a 2D plane strain problem, this is a traction (tx, ty) = (0, q)
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

In [26]:
# Combine boundary conditions
fixed_nodes = fixed_left  # only the left edge is clamped

Assigning material properties to the domain

In [27]:
# --- Material properties ---
E = 100000.0
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")

Material properties: mu=38461.538, kappa=83333.333


Solver step

In [28]:
# Number of incremental load steps
nr_num_steps = 5

# --- Solve with your hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # shape (2, n_nodes)
    connect.T,     # shape (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-10,
    nr_maxit=30,
)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)


Step 0, load factor = 0.200
Iteration 1, Correction=1.000000e+00, Residual=3.178973e-04, tolerance=1.000000e-10
Iteration 2, Correction=1.534369e-03, Residual=2.335916e-03, tolerance=1.000000e-10
Iteration 3, Correction=4.288548e-06, Residual=1.672445e-08, tolerance=1.000000e-10
Iteration 4, Correction=3.575216e-13, Residual=1.050852e-13, tolerance=1.000000e-10
Step 1, load factor = 0.400
Iteration 1, Correction=5.001011e-01, Residual=3.178973e-04, tolerance=1.000000e-10
Iteration 2, Correction=7.665300e-04, Residual=2.340229e-03, tolerance=1.000000e-10
Iteration 3, Correction=2.089098e-06, Residual=1.673326e-08, tolerance=1.000000e-10
Iteration 4, Correction=1.404963e-13, Residual=9.869517e-14, tolerance=1.000000e-10
Step 2, load factor = 0.600
Iteration 1, Correction=3.335097e-01, Residual=3.178973e-04, tolerance=1.000000e-10
Iteration 2, Correction=5.105927e-04, Residual=2.344425e-03, tolerance=1.000000e-10
Iteration 3, Correction=1.355482e-06, Residual=1.674159e-08, tolerance=1.000

Post processing: calculating tip deflection

In [29]:
# --- Compute the tip displacement from the FEA result ---
# We'll pick a node near x=L, y=H/2
tip_node = None
tol = 1e-3
for i, (x, y) in enumerate(coords):
    if abs(x - L) < tol and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break
if tip_node is None:
    raise ValueError("Could not find tip node near x=L, y=H/2.")

tip_disp_y = final_disp[ndof*tip_node + 1]  # the y-component of displacement

In [30]:
# --- Compare with Euler–Bernoulli formula for small deflection ---
mu = material_props[0]
# We'll guess E = 3 mu (plane stress, near incompressible) or 2 mu(1 + nu) for plane strain, etc.
# --- Analytical Solution ---
# For a cantilever beam under uniformly distributed load q:
# Euler–Bernoulli tip deflection: w(L) = q * L^4 / (8 * E * I)
# E_eff = E * (1 - nu) / ((1 + nu) * (1 - 2*nu))
E_eff = E / (1 - nu ** 2.0)
I = H ** 3 / 12.0
w_analytical = q * L ** 4 / (8.0 * E_eff * I)

print(f"Tip node index: {tip_node}, coordinates={coords[tip_node]}")
print(f"Computed tip deflection (y): {tip_disp_y:.6f}")
print(f"Analytical Euler-Bernoulli deflection: {w_analytical:.6f}")

Tip node index: 243, coordinates=[10.   0.5]
Computed tip deflection (y): -0.142029
Analytical Euler-Bernoulli deflection: -0.136500


In [21]:
# --- Evaluate error ---
error = abs(tip_disp_y - w_analytical)
print(f"Absolute error = {error:.6e}")

# --- Plot the mesh with the final deformed shape ---

img_name = "tutorial1.gif"
fname = str(tutorials_dir / img_name)
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

Absolute error = 5.528741e-03


### Animated Displacement Field

![Displacement Animation](tutorial1.gif)